In [11]:
# -*- coding:utf-8 -*-
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from collections import defaultdict
from pyecharts import options as opts
from pyecharts.charts import Graph,Page

%matplotlib inline
plt.rcParams['font.family'] = ['sans-serif']
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus']=False

global minVal,maxVal

class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return super(NpEncoder, self).default(obj)

class GCoeffience(object):
    def __init__(self,fpath,rou):
        self.fpath = fpath
        self.rou = rou
        
        self.rawData=self.getData()
        
        self.node_size=self.rawData.mean(axis=1)*400/self.rawData.mean(axis=1).sum()
        
        self.labels=self.rawData.index.values
        
        self.resData=pd.DataFrame()
        
        self.outData,self.relation=self.getLoopData()
        
    def getData(self):
        data=pd.read_excel(filepath,encoding='utf-8_sig',
                      index_col=0)
        return data
    
    def calCoef(self,rdata):
        dval=minVal+rou*maxVal/(rdata+rou*maxVal)
        return dval
      
    def getLoopData(self):
        global minVal,maxVal
        self.resData.reindex=self.rawData.index
        columns=self.rawData.index.values

        rows,cols=self.rawData.shape
        for i in range(rows):
            normData=self.rawData/self.rawData.iloc[i]
            absData=abs(normData-normData.iloc[i])
            maxVal=max(absData.max())
            minVal=min(absData.min())

            glxs=absData.apply(lambda row:self.calCoef(row))
            self.resData[columns[i]]=glxs.mean(axis=1)
    #         print(glxs.mean(axis=1))
    #     dataw=df.as_matrix()
        dataT=self.resData.values.T
        relation=(self.resData.values+dataT)/2
        newdf=pd.DataFrame(dataT,columns=columns)
        newdf.index=self.rawData.index
#         print(newdf)
        return newdf,np.triu(relation)

    def show_labels(self):
        labls={}
        for (lbl,ns) in zip(self.labels,self.node_size.values):
            if ns >=25:
                labls[lbl]=lbl
        print(labls)
        return labls

    def create_ntx_pds(self):
        self.outData[self.outData<0.56]=0
        G=nx.from_pandas_adjacency(self.outData,create_using = nx.MultiGraph)
        pos=nx.spring_layout(G)
        
        pd_labels=self.show_labels()
        nx.draw_networkx_edges(G,pos,with_labels=False,edge_color='blue',
                              alpha=0.8,width=[float(v['weight']*5) for (r,c,v) in G.edges(data=True)])
        nx.draw_networkx_labels(G,pos,pd_labels,font_size=12)
        nx.draw_networkx(G,pos,node_size=self.node_size.values,with_labels=False)

    def create_ntx_mat(self):
        G=nx.from_numpy_matrix(self.relation,create_using=nx.MultiDiGraph)
        pos=nx.spring_layout(G)
        labls=show_labels(self.labels,self.node_size.values)
    #     nx.draw(G)
    #     nx.draw_networkx_labels(G,pos,labls,font_size=12)
        nx.draw_networkx(G,pos,node_size=self.node_size,
                        with_labels=False)
        
    def show_graph(self):
        plt.axis('off')
        plt.show()

# nx.draw_networkx_edges(G,pos,with_labels=False,edge_color='blue',alpha=0.8,width=[float(v['weight']*5) for (r,c,v) in G.edges(data=True)])


class CyParseModel(object):
    def __init__(self,origPath,outPath):
        self.fpath=origPath
        self.outpath=outPath
        
        self.NodesJson=[]
        self.EdgesJson=[]
        self.CatsJson=[]
        
        self.rawData=self.getData()
        
        self.cy_weight_data,self.hy_weight_data,self.xl_weight_data=self.node_weight()
        
#         self.total_cy_dict,self.total_dl_dict=self.get_cy_data_no(self)
        
    def getData(self):
        data=pd.read_csv(self.fpath,encoding='utf-8_sig',index_col=4)
        prcData=data.drop_duplicates()
        return prcData
    
    def node_weight(self):
        hy_weight_data={}
        cy_weight_data={}
        xl_weight_data={}
        
        min_ptcs=self.rawData[u'企业数量'].groupby([self.rawData['行业大类']]).sum().min()

        cy_data=(self.rawData[u'企业数量'].groupby([self.rawData['产业类别']]).sum()/min_ptcs).apply(lambda row:round(row,3))
        xl_data=(self.rawData[u'企业数量']/min_ptcs).apply(lambda row:round(row,3))
#         xl_data=(self.rawData[u'企业数量'].groupby([self.rawData['行业小类']]).sum()/min_ptcs).apply(lambda row:round(row,3))

        cy_weight_data=dict(cy_data)
        xl_weight_data=dict(xl_data)
        
        hy_weight_data={}
        cy_grp=self.rawData[[u'企业数量',u'行业大类']].groupby([self.rawData['产业类别']])
        for cy,val in cy_grp:
            hy_grp=val[u'企业数量'].groupby(val[u'行业大类']).sum()
            for key,val in dict(hy_grp).items():
                hy_weight_data[(cy,key)]=val
                
#         print(sorted(list(hy_weight_data.values()))[1])
#         print(cy_weight_data)
        return cy_weight_data,hy_weight_data,xl_weight_data

    def get_cy_data_width(self):
        total_cy_dict=defaultdict(dict)
        total_dl_dict=defaultdict(dict)
        
        min_ptcs=sorted(list(self.hy_weight_data.values()))[int(len(self.hy_weight_data.values())/2)]
        hy_grpData=self.rawData[[u'企业数量']].groupby([self.rawData['产业类别'],self.rawData['行业大类']])
        
        cy_total=sum(list(self.cy_weight_data.values()))
        hy_total=sum(list(self.hy_weight_data.values()))
        
        for (cy,dl),val in hy_grpData:
#             print("&&&&&&&&&&%s&&&&&&&&&%s"%(cy,dl))
            tem_dict={}
    
            for (xl,vals) in dict(val).items():
                print(vals)
                pass
#                 tem_dict['nodeSize']=round(vals/min_ptcs,4)
#                 tem_dict['width']=round(vals/self.hy_weight_data[(cy,dl)],4)
#                 total_dl_dict[(cy,dl)][xl]=tem_dict
#             total_dl_dict[(cy,dl)]['width']=round(self.hy_weight_data[(cy,dl)]/self.cy_weight_data[cy],4)
        print(total_dl_dict)
                
                
#             
        
        

    def get_cy_data_width1(self):

        for (dl,xl),val in hy_grpData:
            tem_dict={}
            tem_dict['weight']=round(val.values[0]/min_ptcs,3)
            tem_dict['width']=round(self.xl_weight_data[xl]*10000/xl_total)
            total_dl_dict[dl][xl]=tem_dict
            total_dl_dict[dl]['width']=round(self.hy_weight_data[dl]/hy_total,3)
            del tem_dict
            
        cy_grpData=self.rawData[u'企业数量'].groupby([self.rawData['产业类别'],self.rawData['行业大类']])

        for (cy,cdl),val in cy_grpData:
            tem_dict={}
            tem_dict['weight']=round(val.values[0]/min_ptcs,3)        
            total_cy_dict[cy][cdl]=tem_dict
            total_cy_dict[cy]['width']=round(self.cy_weight_data[cy]/cy_total,3)
            total_cy_dict[cy]['weight']=round(self.cy_weight_data[cy]/min_ptcs,3)

        return total_cy_dict,total_dl_dict
    
    def get_cy_data(self):
        total_cy_dict=defaultdict(dict)
        total_dl_dict=defaultdict(dict)

        min_ptcs=self.rawData[u'企业数量'].groupby([self.rawData['行业大类']]).sum().min()

        hy_grpData=self.rawData[u'企业数量'].groupby([self.rawData['行业大类'],self.rawData['行业小类']])

        for (dl,xl),val in hy_grpData:
            tem_dict={}
            tem_dict['weight']=round(val.values[0]/min_ptcs,4)
            tem_dict['width']=round(xl_weight_data[xl]*10/hy_weight_data[dl],4)
            total_dl_dict[dl][xl]=tem_dict
            total_dl_dict[dl]['weight']=round(self.hy_weight_data[dl]/min_ptcs,4)

            del tem_dict
            
        cy_grpData=self.rawData[u'企业数量'].groupby([self.rawData['产业类别'],self.rawData['行业大类']])

        for (cy,cdl),val in cy_grpData:
            tem_dict={}
            tem_dict['weight']=round(val.values[0]/min_ptcs,3)        
            total_cy_dict[cy][cdl]=tem_dict
            
        return total_cy_dict,total_dl_dict
    
if __name__=="__main__":
    origPath=r'C:\Users\Administrator\Desktop\tpTest\hygs_qxb.csv'
    outPath=r'C:\Users\Administrator\Desktop\tpTest\hy.json'
    cyModel=CyParseModel(origPath,outPath)
    cyModel.get_cy_data_width()

行业小类
农业     4506
林业      137
畜牧业    1455
渔业      739
Name: 企业数量, dtype: int64
行业小类
铁路运输业            7
道路运输业         8339
水上运输业          193
航空运输业           55
管道运输业            1
装卸搬运和运输代理业    2335
仓储业            356
邮政业            289
Name: 企业数量, dtype: int64
行业小类
住宿业    3385
餐饮业    9014
Name: 企业数量, dtype: int64
行业小类
电信、广播电视和卫星传输服务      531
互联网和相关服务            243
软件和信息技术服务业        40811
Name: 企业数量, dtype: int64
行业小类
农、林、牧、渔服务业    257
Name: 企业数量, dtype: int64
行业小类
金属制品、机械和设备修理业    1731
Name: 企业数量, dtype: int64
行业小类
卫生      1136
社会工作     114
Name: 企业数量, dtype: int64
行业小类
居民服务业               3071
机动车、电子产品和日用产品修理业    5713
其他服务业               1773
Name: 企业数量, dtype: int64
行业小类
房地产业     20737
租赁业       2168
商务服务业    75967
Name: 企业数量, dtype: int64
行业小类
批发业    120867
零售业     32512
Name: 企业数量, dtype: int64
行业小类
教育    1711
Name: 企业数量, dtype: int64
行业小类
新闻和出版业                76
广播、电视、电影和影视录音制作业     334
文化艺术业                396
体育                  1933
娱乐业                 3246
Name: 企业数量, dtype: 

In [4]:
# -*- coding:utf-8 -*-
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from collections import defaultdict
import json
from pyecharts import options as opts
from pyecharts.charts import Graph,Page
from pyecharts.charts import Sunburst

%matplotlib inline
plt.rcParams['font.family'] = ['sans-serif']
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus']=False


class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return super(NpEncoder, self).default(obj)

global minVal,maxVal
##灰色关联分析模型计算方法
class GCoeffience(object):
    def __init__(self,fpath,rou):
        self.fpath = fpath
        self.rou = rou
        
        self.rawData=self.getData()
        
        self.node_size=self.rawData.mean(axis=1)*400/self.rawData.mean(axis=1).sum()
        
        self.labels=self.rawData.index.values
        
        self.resData=pd.DataFrame()
        
        self.outData,self.relation=self.getLoopData()
        
    def getData(self):
        data=pd.read_excel(filepath,encoding='utf-8_sig',
                      index_col=0)
        return data
    
    def calCoef(self,rdata):
        dval=minVal+rou*maxVal/(rdata+rou*maxVal)
        return dval
      
    def getLoopData(self):
        global minVal,maxVal
        self.resData.reindex=self.rawData.index
        columns=self.rawData.index.values

        rows,cols=self.rawData.shape
        for i in range(rows):
            normData=self.rawData/self.rawData.iloc[i]
            absData=abs(normData-normData.iloc[i])
            maxVal=max(absData.max())
            minVal=min(absData.min())

            glxs=absData.apply(lambda row:self.calCoef(row))
            self.resData[columns[i]]=glxs.mean(axis=1)
    #         print(glxs.mean(axis=1))
    #     dataw=df.as_matrix()
        dataT=self.resData.values.T
        relation=(self.resData.values+dataT)/2
        newdf=pd.DataFrame(dataT,columns=columns)
        newdf.index=self.rawData.index
#         print(newdf)
        return newdf,np.triu(relation)

    def show_labels(self):
        labls={}
        for (lbl,ns) in zip(self.labels,self.node_size.values):
            if ns >=25:
                labls[lbl]=lbl
        print(labls)
        return labls

    def create_ntx_pds(self):
        self.outData[self.outData<0.56]=0
        G=nx.from_pandas_adjacency(self.outData,create_using = nx.MultiGraph)
        pos=nx.spring_layout(G)
        
        pd_labels=self.show_labels()
        nx.draw_networkx_edges(G,pos,with_labels=False,edge_color='blue',
                              alpha=0.8,width=[float(v['weight']*5) for (r,c,v) in G.edges(data=True)])
        nx.draw_networkx_labels(G,pos,pd_labels,font_size=12)
        nx.draw_networkx(G,pos,node_size=self.node_size.values,with_labels=False)

    def create_ntx_mat(self):
        G=nx.from_numpy_matrix(self.relation,create_using=nx.MultiDiGraph)
        pos=nx.spring_layout(G)
        labls=show_labels(self.labels,self.node_size.values)
    #     nx.draw(G)
    #     nx.draw_networkx_labels(G,pos,labls,font_size=12)
        nx.draw_networkx(G,pos,node_size=self.node_size,
                        with_labels=False)
        
    def show_graph(self):
        plt.axis('off')
        plt.show()

# nx.draw_networkx_edges(G,pos,with_labels=False,edge_color='blue',alpha=0.8,width=[float(v['weight']*5) for (r,c,v) in G.edges(data=True)])

##产业启信宝数据得解析，并得到echart数据之间得联系
class CyParseModel(object):
    def __init__(self,origPath,outPath):
        self.fpath=origPath
        self.outpath=outPath
        
        self.rawData=self.getData()
        
        self.cy_weight_data,self.hy_weight_data,self.xl_weight_data=self.node_weight()
        
        self.total_cy_dict,self.total_dl_dict=self.get_cy_data_link()
        
        self.NodesJson=self.genNode()
        self.EdgesJson=self.genEdges()
        self.CatsJson=self.genCateries()
        
        self.AllData=[self.NodesJson,self.EdgesJson,self.CatsJson]
        
    def getData(self):
        data=pd.read_csv(self.fpath,encoding='utf-8_sig',index_col=4)
        prcData=data.drop_duplicates()
        return prcData
    
    def node_weight(self):
        hy_weight_data={}
        cy_weight_data={}
        xl_weight_data={}
        
#         min_ptcs=self.rawData[u'企业数量'].groupby([self.rawData['行业大类']]).sum().min()

        cy_data=(self.rawData[u'企业数量'].groupby([self.rawData['产业类别']]).sum()).apply(lambda row:round(row,3))
        xl_data=(self.rawData[u'企业数量']).apply(lambda row:round(row,3))
#         xl_data=(self.rawData[u'企业数量'].groupby([self.rawData['行业小类']]).sum()/min_ptcs).apply(lambda row:round(row,3))

        cy_weight_data=dict(cy_data)
        xl_weight_data=dict(xl_data)
        
        hy_weight_data={}
        cy_grp=self.rawData[[u'企业数量',u'行业大类']].groupby([self.rawData['产业类别']])
        for cy,val in cy_grp:
            hy_grp=val[u'企业数量'].groupby(val[u'行业大类']).sum()
            for key,val in dict(hy_grp).items():
                hy_weight_data[(cy,key)]=val

        return cy_weight_data,hy_weight_data,xl_weight_data
    
    def getCyValue(self):
        cy_data=self.rawData[u"行业大类"].groupby(self.rawData['产业类别'])
        return dict(cy_data.count())
    
    def get_cy_data_link(self):
        total_cy_dict=defaultdict(dict)
        total_dl_dict=defaultdict(dict)
        
#         min_ptcs=sorted(list(self.hy_weight_data.values()))[1]
        hy_grpData=self.rawData[u'企业数量'].groupby([self.rawData['产业类别'],self.rawData['行业大类']])
        
        cy_value_wg=self.getCyValue()
    
        cy_total=sum(list(self.cy_weight_data.values()))
        hy_total=sum(list(self.hy_weight_data.values()))
        
        for (cy,dl),val in hy_grpData:
#             print("&&&&&&&&&&%s&&&&&&&&&%s"%(cy,dl))
            for (xl,vals) in val.items():
                tem_dict={}
                tem_dict['nodeSize']=vals
                tem_dict['link']=0
#                 tem_dict['nodeSize']=round(vals/min_ptcs,4)
#                 tem_dict['width']=round(vals/self.hy_weight_data[(cy,dl)],4)
                total_dl_dict[(cy,dl)][xl]=tem_dict
            total_dl_dict[(cy,dl)]['nodeSize']=self.hy_weight_data[(cy,dl)]
            total_dl_dict[(cy,dl)]['link']=val.count()
            
#             total_dl_dict[(cy,dl)]['width']=round(self.hy_weight_data[(cy,dl)]/self.cy_weight_data[cy],4)
            total_cy_dict[cy]['nodeSize']=self.cy_weight_data[cy]
            total_cy_dict[cy]['link']=cy_value_wg[cy]
            total_cy_dict[cy][(cy,dl)]=total_dl_dict[(cy,dl)]
        
#         print(total_cy_dict)
        return total_cy_dict,total_dl_dict

    def get_cy_data_width(self):
        total_cy_dict=defaultdict(dict)
        total_dl_dict=defaultdict(dict)
        
#         min_ptcs=sorted(list(self.hy_weight_data.values()))[1]
        hy_grpData=self.rawData[u'企业数量'].groupby([self.rawData['产业类别'],self.rawData['行业大类']])
        
        cy_total=sum(list(self.cy_weight_data.values()))
        hy_total=sum(list(self.hy_weight_data.values()))
        
        for (cy,dl),val in hy_grpData:
#             print("&&&&&&&&&&%s&&&&&&&&&%s"%(cy,dl))
            for (xl,vals) in dict(val).items():
                tem_dict={}
                tem_dict['nodeSize']=vals
#                 tem_dict['nodeSize']=round(vals/min_ptcs,4)
                tem_dict['width']=round(vals/self.hy_weight_data[(cy,dl)],4)
                total_dl_dict[(cy,dl)][xl]=tem_dict
                total_dl_dict[(cy,dl)]['nodeSize']=self.hy_weight_data[(cy,dl)]
#             total_dl_dict[(cy,dl)]['nodeSize']=round(self.hy_weight_data[(cy,dl)]/min_ptcs,4)
            total_dl_dict[(cy,dl)]['width']=round(self.hy_weight_data[(cy,dl)]/self.cy_weight_data[cy],4)
            total_cy_dict[cy]['nodeSize']=self.cy_weight_data[cy]
#             total_cy_dict[cy]['nodeSize']=round(self.cy_weight_data[cy]/min_ptcs,4)
            total_cy_dict[cy]['width']=round(self.cy_weight_data[cy]/cy_total,4)
            total_cy_dict[cy][(cy,dl)]=total_dl_dict[(cy,dl)]
        
#         print(total_cy_dict)
        return total_cy_dict,total_dl_dict
     
    def genCateries(self):
        catsList=[]
        keys=self.total_dl_dict.keys()
        
        temp={}
        i=0
        for (cy,dl) in keys:
            if cy not in temp.keys():
                temp[cy]=i
                catsList.append({"name":cy})
            if dl not in temp.keys():
                temp[dl]=i+1
                catsList.append({"name":dl})
        del temp
        return catsList
    
    def genEdges(self):
        edgesList=[]
        tem_dict={}
        i=0
        edgesList.append({"source":u"第一产业","target":u"第三产业"})
        edgesList.append({"source":u"第二产业","target":u"第三产业"})
        for ((cy,dl)),dvals in self.total_dl_dict.items():
            if (cy,dl) not in tem_dict.keys():
                edgesList.append({"source":cy,"target":dl})
                tem_dict[(cy,dl)]=i
            for xl,cvals in dvals.items():
                if (dl,xl) not in tem_dict.keys() and xl!="link" and xl !="nodeSize":
                    edgesList.append({"source":dl,"target":xl})
                    tem_dict[(dl,xl)]=i+1
        del tem_dict
        return edgesList
    
    def getLevel(self,k):
        if k>100000:
            return 25
        elif k>=50000:
            return 18
        elif k>=10000:
            return 11
        elif k>=4000:
            return 9
        elif k>=2000:
            return 6
        else:
            return 2
    
    def getCyTempDict(self,cy,dl):
        tem_dict={}
        tem_dict["name"]=cy
#         tem_dict["symbolSize"]=self.total_cy_dict[cy]['nodeSize']
        if cy==u"第一产业":
            tem_dict["symbolSize"]=12
        elif cy==u"第二产业":
            tem_dict["symbolSize"]=26
        else:
            tem_dict["symbolSize"]=30
#         tem_dict["symbolSize"]=self.getLevel(self.total_cy_dict[cy]['nodeSize'])
        tem_dict["draggable"]="False"
        tem_dict["value"]=self.total_cy_dict[cy]['link']
#         tem_dict["value"]=tem_dict["symbolSize"]
        tem_dict["category"]=cy
        if self.getLevel(tem_dict["symbolSize"])>7:
            tem_dict["label"]={"normal":{"show":"True"}}
        else:
            tem_dict["label"]={"normal":{"show":"False"}}
        return tem_dict

    def getHyTempDict(self,cy,dl):
        tem_dict={}
        tem_dict["name"]=dl
        tem_dict["symbolSize"]=self.getLevel(self.total_dl_dict[(cy,dl)]['nodeSize'])
        tem_dict["draggable"]="False"
        tem_dict["value"]=self.total_dl_dict[(cy,dl)]['link']
#             tem_dict["value"]=int(self.total_dl_dict[(cy,dl)]['width']*10)
        tem_dict["category"]=cy
        if tem_dict["symbolSize"]>=7:
            tem_dict["label"]={"normal":{"show":"True"}}
        else:
            tem_dict["label"]={"normal":{"show":"False"}}
#         tem_dict["lable"]={"normal":{"show":"True"}}
        return tem_dict
    
    def getXlTempDict(self,cy,dl,xl):
        tem_dict={}
        tem_dict["name"]=xl
        tem_dict["symbolSize"]=self.getLevel(self.total_dl_dict[(cy,dl)][xl]['nodeSize'])
        tem_dict["draggable"]="False"
        tem_dict["value"]=0
        if tem_dict["symbolSize"]>7:
            tem_dict["label"]={"normal":{"show":"True"}}
#         if self.total_dl_dict[(cy,dl)][xl]['link']*10<1:
#             tem_dict["value"]=1
#         else:
#             tem_dict["value"]=int(self.total_dl_dict[(cy,dl)][xl]['width']*10)
        tem_dict["category"]=dl
        return tem_dict
    
    def genNode(self):
        nodesList=[]
        cy_dict={}
        hy_dict={}
        i=0
        for ((cy,dl),dval) in self.total_dl_dict.items():
            if cy not in cy_dict:
                tem_dict=self.getCyTempDict(cy,dl)
                i=i+1
                cy_dict[cy]=i
                nodesList.append(tem_dict)
            if dl not in cy_dict:
                dl_dict=self.getHyTempDict(cy,dl)
                i=i+1
                cy_dict[dl]=i
                nodesList.append(dl_dict)
                del dl_dict
            for xl,xval in dval.items():
                if xl=="nodeSize" or xl=="link":
                    continue
                if xl not in cy_dict:
                    xl_dict=self.getXlTempDict(cy,dl,xl)
                    i=i+1
                    cy_dict[xl]=i
                    nodesList.append(xl_dict)
                    del xl_dict
        print(len(nodesList))
        return nodesList
    
    def genSunBurstJson(self):
        pass
    
    
    def genJson(self):
        with open(self.outpath,'w',encoding='utf-8') as f:
            json.dump(self.AllData,f,ensure_ascii=False,cls=NpEncoder)
        
class CreateGraph():
    def __init__(self,jsonPath):
        
        self.jpath=jsonPath
        
    
    def graph_weibo(self) ->Graph:
        with open(self.jpath, "r", encoding="utf-8_sig") as f:
            j = json.load(f)
            nodes, links, categories = j
        c = (
            Graph(init_opts=opts.InitOpts(width="1000px", height="900px"))
            .add(
                "",
                nodes,
                links,
                categories,
                repulsion=100,
                linestyle_opts=opts.LineStyleOpts(curve=0.2),
                label_opts=opts.LabelOpts(is_show=False),
            )
            .set_global_opts(
                legend_opts=opts.LegendOpts(is_show=False),
                title_opts=opts.TitleOpts(title="产业 — 行业关系图"),
            )
        )
        return c
    
    def graph_les_miserables() -> Graph:
        with open('self.lspath', "r",encoding="utf-8") as f:
            j = json.load(f)
            nodes = j["nodes"]
            links = j["links"]
            categories = j["categories"]

        c = (
             Graph(init_opts=opts.InitOpts(width="1000px", height="600px")).add("",
                nodes=nodes,
                links=links,
                categories=categories,
                layout="circular",
                is_rotate_label=True,
                linestyle_opts=opts.LineStyleOpts(color="source", curve=0.3),
                label_opts=opts.LabelOpts(position="right"),).set_global_opts(
                title_opts=opts.TitleOpts(title="Graph-Les Miserables"),
                legend_opts=opts.LegendOpts(
                    orient="vertical", pos_left="2%", pos_top="20%"
                ),)
        )
        return c
    
    def graph_npm_dependencies(self) -> Graph:
        with open('self.nmpath', "r", encoding="utf-8") as f:
            j = json.load(f)
        nodes = [
            {
                "x": node["x"],
                "y": node["y"],
                "id": node["id"],
                "name": node["label"],
                "symbolSize": node["size"],
                "itemStyle": {"normal": {"color": node["color"]}},
            }
            for node in j["nodes"]
        ]

        edges = [
            {"source": edge["sourceID"], "target": edge["targetID"]} for edge in j["edges"]
        ]

        c = (
            Graph(init_opts=opts.InitOpts(width="1000px", height="600px"))
            .add(
                "",
                nodes=nodes,
                links=edges,
                layout="none",
                label_opts=opts.LabelOpts(is_show=False),
                linestyle_opts=opts.LineStyleOpts(width=0.5, curve=0.3, opacity=0.7),
            )
            .set_global_opts(title_opts=opts.TitleOpts(title="Graph-NPM Dependencies"))
            .set_series_opts(label_opts=opts.LabelOpts(formatter="{b}"))
        )
        return c
    
    def sunburst_base(self) -> Sunburst:
        data=[
            opts.SunburstItem(
                name='Grandpa',
                children=[
                    opts.SunburstItem(
                        name='uncle Leo',
                        value=15,
                        children=[
                            opts.SunburstItem(name='jack',value=2),
                            opts.SunburstItem(name='Margy',value=5,
                                             children=[opts.SunburstItem(name='Jackson',value=2)]),
                            opts.SunburstItem(name='Ben',value=4)
                        ],
                    ),
                    opts.SunburstItem(name='Father',value=10,
                                     children=[
                                         opts.SunburstItem(name='me',value=5),
                                         opts.SunburstItem(name='b pter',value=2)
                                     ]),
                ],
            ),
                opts.SunburstItem(
                name="Nancy",
                children=[
                    opts.SunburstItem(
                        name="Uncle Nike",
                        children=[
                            opts.SunburstItem(name="Cousin Betty", value=1),
                            opts.SunburstItem(name="Cousin Jenny", value=2),
                        ],
                    )
                ],
            ),
        ]
        c=(
            Sunburst()
            .add(series_name="", data_pair=data, radius=[0, "90%"])
            .set_global_opts(title_opts=opts.TitleOpts(title="Sunburst-基本示例"))
            .set_series_opts(label_opts=opts.LabelOpts(formatter="{b}"))
        )
        return c
    
    def sunburst_official(self) -> Sunburst:
        with open(r'C:\Users\Administrator\Desktop\hsgl\drink.json','r',encoding='utf-8') as f:
            j=json.load(f)
        gph=(
            Sunburst(init_opts=opts.InitOpts(width='1000px',height='600px'))
            .add(
                '',data_pair=j,highlight_policy='ancestor',
                radius=[0,'95%'],sort_='null',
                levels=[
                    {},
                    {'r0':'15%','r':'35%','itemStyle':{'borderWidth':2},
                     'lable':{'rotate':'tangential'},},
                    {'r0':'35%','r':'70%','lable':{'align':'right'}},
                    {
                        'ro':'70%',
                        'r':'72%',
                        'label':{'position':'outside','padding':3,'silent':False},
                        'itemStyle':{'borderWidth':3},
                    },
                ],
            )
            .set_global_opts(title_opts=opts.TitleOpts(title="Sunburst-官方示例"))
            .set_series_opts(label_opts=opts.LabelOpts(formatter="{b}"))
        )
        return gph 


if __name__=="__main__":
    origPath=r'C:\Users\Administrator\Desktop\tpTest\hygs_qxb.csv'
    outPath=r'C:\Users\Administrator\Desktop\tpTest\cyjieguo4.json'
    cyModel=CyParseModel(origPath,outPath)
    print(cyModel.total_dl_dict)
    cyModel.genJson()
    cgraph=CreateGraph(outPath)
    cg=cgraph.graph_weibo()
    cg.render(r'C:\Users\Administrator\Desktop\tpTest\产业关系图2.html')
    print('success')

110
defaultdict(<class 'dict'>, {('第一产业', '农、林、牧、渔业'): {'农业': {'nodeSize': 4506, 'link': 0}, '林业': {'nodeSize': 137, 'link': 0}, '畜牧业': {'nodeSize': 1455, 'link': 0}, '渔业': {'nodeSize': 739, 'link': 0}, 'nodeSize': 6837, 'link': 4}, ('第三产业', '交通运输、仓储和邮政业'): {'铁路运输业': {'nodeSize': 7, 'link': 0}, '道路运输业': {'nodeSize': 8339, 'link': 0}, '水上运输业': {'nodeSize': 193, 'link': 0}, '航空运输业': {'nodeSize': 55, 'link': 0}, '管道运输业': {'nodeSize': 1, 'link': 0}, '装卸搬运和运输代理业': {'nodeSize': 2335, 'link': 0}, '仓储业': {'nodeSize': 356, 'link': 0}, '邮政业': {'nodeSize': 289, 'link': 0}, 'nodeSize': 11575, 'link': 8}, ('第三产业', '住宿和餐饮业'): {'住宿业': {'nodeSize': 3385, 'link': 0}, '餐饮业': {'nodeSize': 9014, 'link': 0}, 'nodeSize': 12399, 'link': 2}, ('第三产业', '信息传输、软件和信息技术服务业'): {'电信、广播电视和卫星传输服务': {'nodeSize': 531, 'link': 0}, '互联网和相关服务': {'nodeSize': 243, 'link': 0}, '软件和信息技术服务业': {'nodeSize': 40811, 'link': 0}, 'nodeSize': 41585, 'link': 3}, ('第三产业', '农、林、牧、渔业服务'): {'农、林、牧、渔服务业': {'nodeSize': 257, 'link': 0}, 'nodeS

In [3]:
# -*- coding:utf-8 -*-
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from collections import defaultdict
import json
from pyecharts import options as opts
from pyecharts.charts import Graph,Page
from pyecharts.charts import Sunburst

%matplotlib inline
plt.rcParams['font.family'] = ['sans-serif']
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus']=False

global minVal,maxVal
maxVal=0.0
minVal=0.0

class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return super(NpEncoder, self).default(obj)


##灰色关联分析模型计算方法
class GCoeffience(object):
    def __init__(self,fpath,rou):
        self.fpath = fpath
        self.rou = rou
        
        self.rawData=self.getData()
        
        self.node_size=self.rawData.mean(axis=1)*400/self.rawData.mean(axis=1).sum()
        
        self.labels=self.rawData.index.values
        
        self.resData=pd.DataFrame()
        
        self.outData,self.relation=self.getLoopData()
        
    def getData(self):
        data=pd.read_excel(filepath,encoding='utf-8_sig',
                      index_col=0)
        return data
    
    def calCoef(self,rdata):
        dval=minVal+rou*maxVal/(rdata+rou*maxVal)
        return dval
      
    def getLoopData(self):
        global minVal,maxVal
        self.resData.reindex=self.rawData.index
        columns=self.rawData.index.values

        rows,cols=self.rawData.shape
        for i in range(rows):
            normData=self.rawData/self.rawData.iloc[i]
            absData=abs(normData-normData.iloc[i])
            maxVal=max(absData.max())
            minVal=min(absData.min())

            glxs=absData.apply(lambda row:self.calCoef(row))
            self.resData[columns[i]]=glxs.mean(axis=1)
    #         print(glxs.mean(axis=1))
    #     dataw=df.as_matrix()
        dataT=self.resData.values.T
        relation=(self.resData.values+dataT)/2
        newdf=pd.DataFrame(dataT,columns=columns)
        newdf.index=self.rawData.index
#         print(newdf)
        return newdf,np.triu(relation)

    def show_labels(self):
        labls={}
        for (lbl,ns) in zip(self.labels,self.node_size.values):
            if ns >=25:
                labls[lbl]=lbl
        print(labls)
        return labls

    def create_ntx_pds(self):
        self.outData[self.outData<0.56]=0
        G=nx.from_pandas_adjacency(self.outData,create_using = nx.MultiGraph)
        pos=nx.spring_layout(G)
        
        pd_labels=self.show_labels()
        nx.draw_networkx_edges(G,pos,with_labels=False,edge_color='blue',
                              alpha=0.8,width=[float(v['weight']*5) for (r,c,v) in G.edges(data=True)])
        nx.draw_networkx_labels(G,pos,pd_labels,font_size=12)
        nx.draw_networkx(G,pos,node_size=self.node_size.values,with_labels=False)

    def create_ntx_mat(self):
        G=nx.from_numpy_matrix(self.relation,create_using=nx.MultiDiGraph)
        pos=nx.spring_layout(G)
        labls=show_labels(self.labels,self.node_size.values)
    #     nx.draw(G)
    #     nx.draw_networkx_labels(G,pos,labls,font_size=12)
        nx.draw_networkx(G,pos,node_size=self.node_size,
                        with_labels=False)
        
    def show_graph(self):
        plt.axis('off')
        plt.show()
        
    

if __name__=="__main__":
    filepath=r'C:\Users\Administrator\Desktop\hsgl\rawdata.xlsx'
    rou=0.5
    gcoffience=GCoeffience(filepath,rou)
    print(gcoffience.outData)

                农林牧渔业        工业       建筑业    批发和零售业  交通运输、仓储和邮政业    住宿和餐饮业  \
农林牧渔业        1.000000  0.380119  0.814009  0.778028     0.943057  0.987920   
工业           0.337052  1.000000  0.371526  0.381130     0.345464  0.336577   
建筑业          0.771287  0.353532  1.000000  0.929324     0.821862  0.768782   
批发和零售业       0.721436  0.352862  0.928955  1.000000     0.765491  0.718544   
交通运输、仓储和邮政业  0.942133  0.368422  0.843096  0.801908     1.000000  0.937219   
住宿和餐饮业       0.986589  0.355912  0.795912  0.758316     0.935392  1.000000   
金融业          0.851921  0.390608  0.935591  0.884941     0.905128  0.850062   
房地产业         0.892633  0.384957  0.893215  0.847578     0.947121  0.889703   
其他服务业        0.415358  0.404430  0.499622  0.527829     0.434776  0.414258   

                  金融业      房地产业     其他服务业  
农林牧渔业        0.858281  0.895852  0.558499  
工业           0.361785  0.353921  0.499979  
建筑业          0.925872  0.873030  0.577090  
批发和零售业       0.867903  0.816117  0.593380  